<a href="https://colab.research.google.com/github/AshishGusain17/models/blob/master/ipynb/local/videos-tf2-obj-det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --user Cython
!pip install --user contextlib2
!pip install --user pillow
!pip install --user lxml
!pip install --user jupyter
!pip install --user matplotlib

In [24]:
%cd research
!python setup.py build
!python setup.py install
%cd ..

C:\Users\hp\Desktop\tf-obj-detection\research


In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/faster_rcnn_resnet101_coco_2018_01_28' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28' '/content/models/research/object_detection'

!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/training' '/content/models/research/object_detection'
!cp -r --recursive '/content/drive/My Drive/colab_obj_detection/images' '/content/models/research/object_detection'

In [0]:

%cd /content/models/research
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
%cd object_detection/legacy
%cd ..
%cd ..
%cd ..
%cd ..
%cd ..

/content/models/research
env: PYTHONPATH=/content/models/research:/content/models/research/slim
/content/models/research/object_detection/legacy
/content/models/research/object_detection
/content/models/research
/content/models
/content
/


In [4]:
%ls
%cd research


 Volume in drive C is Windows
 Volume Serial Number is 68D3-0A2F

 Directory of C:\Users\hp\Desktop\tf-obj-detection

22-04-2020  16:44    <DIR>          .
22-04-2020  16:44    <DIR>          ..
22-04-2020  01:18    <DIR>          .github
22-04-2020  01:18             1,398 .gitignore
22-04-2020  01:18                 0 .gitmodules
22-04-2020  01:18               347 AUTHORS
22-04-2020  01:18             2,525 CODEOWNERS
22-04-2020  01:18             1,476 CONTRIBUTING.md
22-04-2020  16:45    <DIR>          ipynb
22-04-2020  01:18             2,093 ISSUE_TEMPLATE.md
22-04-2020  01:18             1,146 ISSUES.md
22-04-2020  01:18            11,608 LICENSE
22-04-2020  01:18    <DIR>          official
22-04-2020  01:18             1,279 README.md
22-04-2020  01:32    <DIR>          research
22-04-2020  01:18                 0 WORKSPACE
              10 File(s)         21,872 bytes
               6 Dir(s)  734,725,783,552 bytes free
C:\Users\hp\Desktop\tf-obj-detection\research


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import os
import pathlib
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [0]:
%ls
def load_model(model_name):

  model_dir =  "object_detection/faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12/saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [0]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('/content/models/research/object_detection/test_images/image1.jpg'),
 PosixPath('/content/models/research/object_detection/test_images/image2.jpg')]

In [32]:
model_name = 'faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12'
detection_model = load_model(model_name)
# http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [33]:
print(detection_model.inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [34]:
detection_model.output_dtypes

{'detection_classes': tf.float32,
 'num_detections': tf.float32,
 'detection_boxes': tf.float32,
 'detection_scores': tf.float32}

In [35]:
detection_model.output_shapes

{'detection_classes': TensorShape([None, 100]),
 'num_detections': TensorShape([None]),
 'detection_boxes': TensorShape([None, 100, 4]),
 'detection_scores': TensorShape([None, 100])}

In [0]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [0]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  # image_np = np.array(Image.open(image_path))
  image_np = np.array(image_path)
  print(image_np.shape)

  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  # display(Image.fromarray(image_np))
  return image_np

In [0]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

In [0]:
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time
from sklearn.metrics import pairwise

cap=cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out1 = cv2.VideoWriter('i.avi', fourcc, 3.0, (int(cap.get(3)),int(cap.get(4))))

time.sleep(2.0)

ctt = 0
while True:
    (grabbed, frame) = cap.read()
    print('frame',frame.shape)

    # resize the frame, blur it, and convert it to the HSV color space
    # frame = imutils.resize(frame, width=600)
    # print('frame_resize',frame.shape)
    print(ctt)
    ctt = ctt + 1
    frame=show_inference(detection_model, frame)


    cv2.imshow("version", frame)
    out1.write(frame)
    key=cv2.waitKey(100)
    if key & 0xFF == ord("q"):
        break
cap.release()
out1.release()
cv2.destroyAllWindows() 